In [1]:
## exercise 1
# a) false
# b) false
# c) false
# d) false
# e) false
# f) false
# g) true
# h) false
# i) true
# j) false

In [47]:
## exercise 2
# random search CV would be used when you have little time to tune your model with a huge dataset. random search CV consumes less time and resources by
# randomly picking hyper-parameters, making a set, and training it on the model.

In [ ]:
## exercise 3
# stacking is an ensemble machine learning algorithm that can utilize the capabilities of previously trained models to make predictions. stacking is also
# referred to as 'blending' and it uses a strategy that's more ideal than cross-validation's winner takes all approach from the base learner's predictions.

In [ ]:
## exercise 4
# yes, you can average the predictions of each model and utililze an ensemble model to generate even more accurate predictions.

In [ ]:
## exercise 5
# there are many hyper-parameters within the XGB framework that help reduce/prevent overfitting. i would try and re-tune the number of 
# n_estimators because too many trees can cause overfitting. if this doesn't work, i would try and tune either the learning_rate, max_depth,
# colsample_bytree because they all help with reducing overfitting on models.

In [ ]:
## exercise 6
# a) support vector machine

In [ ]:
## exercise 7
# XGB with random forest as a base learner uses parallel trees to fit a model. with this parameter, you can decide how many parallel trees you want 
# to have in each round of boosting. each set of parallel trees will provide you a score after each round of training.

In [ ]:
## exercise 8
# b) performance on the validation dataset

In [ ]:
## exercise 9
# a) 450 trees because at this point on the graph, performance for testing is the highest with 450 trees and is still fairly high for the training dataset.
# b) overfitting is the cause of the difference in F1 perfomance between train and testing results. the training model is performing well, but the model does not
# preform as well on the testing dataset.

In [ ]:
## exercise 10
# the difference between homogenous and heterogenous ensembling techniques is that homogenous ensembling uses the same model but will resample the data.
# in other words, homogenous ensembling uses different subsets of the data on the same base learner, whereas, heterogenous ensemnbles utilize different
# base learners

In [ ]:
## exercise 11
# d) a & c

In [ ]:
## exercise 12
# Grdisearch CV is a good approach to this problem, however, I would change the number of folds.
# using a cross validation value of 5 with only 4 labels as fradulent means that one of the folds
# will not have an observation labeled as fradulent. this would not produce a good predictive model (because they don't have anything to test on). 
# the max number of fold number i would use for this is 4 so that each fold has at least one fradulent label.

In [ ]:
## exercise 13
# a) high values of learning_rate, max_depth, subsample, and colsample_by tree may cause overfitting.
# b) min_child_weight is a hyperparameter that may help prevent overfitting if we increase their values.

In [ ]:
## exercise 14
# to manually optimize the number of trees, data scientists utilize early stopping rounds. this technique allows the model to stop after performance doesn't improve
# after a certain number of rounds.

In [ ]:
## exercise 15
# a) this model is addressing a regression problem because it's being evaluated with reg.squarederror.
# b) the max_depth value is quite large. well-preforming XGB regression models typically have low values for max_depth. increasing min_child_weight
# to at least 5 will also help overfit models.

In [ ]:
## exercise 16
# if had an overfit random forest model, i would adjust the number of trees used in the model, the depth of those trees, and the max_samples_leaf
# parameter. these hyper-parameters deal with sampling and how much data is allowed in a tree.
# to address underfitting, i would increase the number of trees and the amount of splits each tree can utilize.

In [2]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [5]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler

import optuna

from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.ensemble import AdaBoostRegressor

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'Customers.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
customers = pd.read_csv(file_content_stream)
customers.head()

,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,1,Male,19,15000,39,Healthcare,1,4
1,2,Male,21,35000,81,Engineer,3,3
2,3,Female,20,86000,6,Engineer,1,1
3,4,Female,23,59000,77,Lawyer,0,2
4,5,Female,31,38000,40,Entertainment,2,6


In [6]:
## removing CustomerID
customers = customers.drop(columns = 'CustomerID', axis = 1)

In [7]:
customers.head()

,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,Male,19,15000,39,Healthcare,1,4
1,Male,21,35000,81,Engineer,3,3
2,Female,20,86000,6,Engineer,1,1
3,Female,23,59000,77,Lawyer,0,2
4,Female,31,38000,40,Entertainment,2,6


In [8]:
#customers = customers.dropna

In [9]:
## splitting original dataframe into 80% training and 20% testing
train, test = train_test_split(customers, test_size = 0.2)

train.shape

In [10]:
test.shape

(400, 7)

### top variables from exam 1
##### age_income
##### Annual Income ($)
##### income_familysize
##### age_workexp
##### Profession_Artist
##### Gender_Female
##### Profession_Healthcare

### engineering features from exam 1

In [11]:
## annual income / family size
train['income_familysize'] = train['Annual Income ($)'] / train['Family Size']

test['income_familysize'] = test['Annual Income ($)'] / test['Family Size']

## annual income / age
train['age_income'] = train['Annual Income ($)'] / (train['Age'] + 1e-6)

test['age_income'] = test['Annual Income ($)'] / (test['Age'] + 1e-6)

## age^work experience
train['age_workexp'] = (train['Age'])^(train['Work Experience'])

test['age_workexp'] = (test['Age'])^(test['Work Experience'])

In [12]:
## dummy variables for 'Gender' and 'Profession'
train = pd.concat([train.drop(columns = ['Gender', 'Profession'], axis = 1),
                      pd.get_dummies(train[['Gender', 'Profession']])], axis = 1)

test = pd.concat([test.drop(columns = ['Gender', 'Profession'], axis = 1),
                      pd.get_dummies(test[['Gender', 'Profession']])], axis = 1)

In [13]:
## defining input and target variables
x = train[['age_income', 'Annual Income ($)', 'income_familysize', 'age_workexp',
           'Profession_Artist', 'Gender_Female', 'Profession_Healthcare']]
y = train['Spending Score (1-100)']

## defining input and target variables on testing dataset
x_final = test[['age_income', 'Annual Income ($)', 'income_familysize', 'age_workexp',
           'Profession_Artist', 'Gender_Female', 'Profession_Healthcare']]
y_final = test['Spending Score (1-100)']

## scaling the data
scaler = MinMaxScaler()

## scaling inputs on training and testing dataset
x = pd.DataFrame(scaler.fit_transform(x))
x_final = pd.DataFrame(scaler.fit_transform(x_final))

In [14]:
train.head()

,Age,Annual Income ($),Spending Score (1-100),Work Experience,Family Size,income_familysize,age_income,age_workexp,Gender_Female,Gender_Male,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing
1051,17,116822,84,1,3,38940.666667,6871.881949,16,0,1,0,1,0,0,0,0,0,0,0
1701,55,103933,31,6,3,34644.333333,1889.690875,49,0,1,0,0,0,0,1,0,0,0,0
1983,52,55395,41,10,1,55395.000000,1065.288441,62,1,0,0,0,0,0,0,0,0,0,1
611,7,74928,12,0,6,12488.000000,10703.998471,7,1,0,1,0,0,0,0,0,0,0,0
45,24,3000,65,4,2,1500.000000,124.999995,28,1,0,0,0,0,0,0,0,0,1,0


### Model 1
***
LGBM

In [15]:
## OPTUNA ##
def objective(trial):
    lgbm_params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000, step = 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
    }

    model = LGBMRegressor(**lgbm_params)
    rmse_list_lgbm = []
    
    kf = KFold(n_splits = 3, shuffle = True)
    
    for train_ix, val_ix in kf.split(x, y):

        ## splitting the data into train and validation
        x_train_lgbm, x_val_lgbm = x.iloc[train_ix], x.iloc[val_ix]
        y_train_lgbm, y_val_lgbm = y.iloc[train_ix], y.iloc[val_ix]

        ## fitting model
        model.fit(x_train_lgbm, y_train_lgbm)
        
        ## predicting on validation
        y_pred_lgbm = model.predict(x_val_lgbm)
        
        ## appending to list
        rmse_list_lgbm.append(mean_squared_error(y_val_lgbm, y_pred_lgbm, squared = False))
    rmse_lgbm = np.mean(rmse_list_lgbm)
    return rmse_lgbm

study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective, n_trials=100)
print('The best LightGBM paramters based on Optuna tuning are', study_lgbm.best_params)

[I 2023-04-03 14:39:24,251] A new study created in memory with name: no-name-c3136c41-d81e-4317-bada-8d1b0e16b222
[I 2023-04-03 14:39:24,390] Trial 0 finished with value: 27.93897499484139 and parameters: {'n_estimators': 1000, 'learning_rate': 0.027830085010101802, 'num_leaves': 90, 'max_depth': 6, 'min_data_in_leaf': 7100, 'lambda_l1': 40, 'lambda_l2': 95, 'min_gain_to_split': 5.051728765566618}. Best is trial 0 with value: 27.93897499484139.


[LightGBM] [Warning] lambda_l1 is set=40, reg_alpha=0.0 will be ignored. Current value: lambda_l1=40
[LightGBM] [Warning] min_data_in_leaf is set=7100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7100
[LightGBM] [Warning] min_gain_to_split is set=5.051728765566618, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.051728765566618
[LightGBM] [Warning] lambda_l2 is set=95, reg_lambda=0.0 will be ignored. Current value: lambda_l2=95
[LightGBM] [Warning] lambda_l1 is set=40, reg_alpha=0.0 will be ignored. Current value: lambda_l1=40
[LightGBM] [Warning] min_data_in_leaf is set=7100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7100
[LightGBM] [Warning] min_gain_to_split is set=5.051728765566618, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.051728765566618
[LightGBM] [Warning] lambda_l2 is set=95, reg_lambda=0.0 will be ignored. Current value: lambda_l2=95
[LightGBM] [Warning] lambda_l1 is set=

[I 2023-04-03 14:39:24,502] Trial 1 finished with value: 27.961525666468983 and parameters: {'n_estimators': 400, 'learning_rate': 0.20990375088310628, 'num_leaves': 41, 'max_depth': 8, 'min_data_in_leaf': 8800, 'lambda_l1': 70, 'lambda_l2': 5, 'min_gain_to_split': 11.890990631074928}. Best is trial 0 with value: 27.93897499484139.
[I 2023-04-03 14:39:24,537] Trial 2 finished with value: 27.994538925215036 and parameters: {'n_estimators': 150, 'learning_rate': 0.12469829335228819, 'num_leaves': 68, 'max_depth': 7, 'min_data_in_leaf': 7500, 'lambda_l1': 65, 'lambda_l2': 85, 'min_gain_to_split': 9.34977581998737}. Best is trial 0 with value: 27.93897499484139.
[I 2023-04-03 14:39:24,562] Trial 3 finished with value: 27.991363642210697 and parameters: {'n_estimators': 50, 'learning_rate': 0.20225109031284869, 'num_leaves': 55, 'max_depth': 14, 'min_data_in_leaf': 5000, 'lambda_l1': 0, 'lambda_l2': 75, 'min_gain_to_split': 8.890137130214837}. Best is trial 0 with value: 27.93897499484139.


[LightGBM] [Warning] lambda_l1 is set=70, reg_alpha=0.0 will be ignored. Current value: lambda_l1=70
[LightGBM] [Warning] min_data_in_leaf is set=8800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8800
[LightGBM] [Warning] min_gain_to_split is set=11.890990631074928, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11.890990631074928
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[LightGBM] [Warning] lambda_l1 is set=65, reg_alpha=0.0 will be ignored. Current value: lambda_l1=65
[LightGBM] [Warning] min_data_in_leaf is set=7500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7500
[LightGBM] [Warning] min_gain_to_split is set=9.34977581998737, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=9.34977581998737
[LightGBM] [Warning] lambda_l2 is set=85, reg_lambda=0.0 will be ignored. Current value: lambda_l2=85
[LightGBM] [Warning] lambda_l1 is set=65

[I 2023-04-03 14:39:24,971] Trial 5 finished with value: 27.984385622373548 and parameters: {'n_estimators': 950, 'learning_rate': 0.28915099567951863, 'num_leaves': 86, 'max_depth': 16, 'min_data_in_leaf': 500, 'lambda_l1': 100, 'lambda_l2': 30, 'min_gain_to_split': 4.499834569391395}. Best is trial 0 with value: 27.93897499484139.
[I 2023-04-03 14:39:25,089] Trial 6 finished with value: 27.948770996208935 and parameters: {'n_estimators': 950, 'learning_rate': 0.05401109016822929, 'num_leaves': 91, 'max_depth': 17, 'min_data_in_leaf': 6900, 'lambda_l1': 15, 'lambda_l2': 50, 'min_gain_to_split': 3.4984358296291673}. Best is trial 0 with value: 27.93897499484139.
[I 2023-04-03 14:39:25,118] Trial 7 finished with value: 27.94805486895866 and parameters: {'n_estimators': 50, 'learning_rate': 0.23302531792751044, 'num_leaves': 30, 'max_depth': 16, 'min_data_in_leaf': 2400, 'lambda_l1': 65, 'lambda_l2': 60, 'min_gain_to_split': 4.75522852799151}. Best is trial 0 with value: 27.9389749948413

...

[LightGBM] [Warning] lambda_l1 is set=15, reg_alpha=0.0 will be ignored. Current value: lambda_l1=15
[LightGBM] [Warning] min_data_in_leaf is set=9200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9200
[LightGBM] [Warning] min_gain_to_split is set=4.262212692048333, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.262212692048333
[LightGBM] [Warning] lambda_l2 is set=25, reg_lambda=0.0 will be ignored. Current value: lambda_l2=25
[LightGBM] [Warning] lambda_l1 is set=15, reg_alpha=0.0 will be ignored. Current value: lambda_l1=15
[LightGBM] [Warning] min_data_in_leaf is set=9200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9200
[LightGBM] [Warning] min_gain_to_split is set=4.262212692048333, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.262212692048333
[LightGBM] [Warning] lambda_l2 is set=25, reg_lambda=0.0 will be ignored. Current value: lambda_l2=25
[LightGBM] [Warning] lambda_l1 is set=

[I 2023-04-03 14:39:36,679] Trial 95 finished with value: 27.94230347231012 and parameters: {'n_estimators': 900, 'learning_rate': 0.0496885927625599, 'num_leaves': 90, 'max_depth': 9, 'min_data_in_leaf': 9400, 'lambda_l1': 15, 'lambda_l2': 10, 'min_gain_to_split': 3.492057871824224}. Best is trial 34 with value: 27.915220176491626.
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:132: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:149: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)
[I 2023-04-03 14:39:36,825] Trial 96 finished with value: 27.951134998398903 and parameters: {'n_estimators': 900, 'learning_rate': 0.07811504592552684, 'num_leaves': 92, 'max_depth': 10, 'min_data_in_leaf': 9900, 'lambda_l1': 5, 'lambda_l2': 25, 'min_gain_to_split': 4.327382199822327}. Best is trial 34 wi

[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] min_data_in_leaf is set=9900, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9900
[LightGBM] [Warning] min_gain_to_split is set=4.327382199822327, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.327382199822327
[LightGBM] [Warning] lambda_l2 is set=25, reg_lambda=0.0 will be ignored. Current value: lambda_l2=25
[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] min_data_in_leaf is set=9900, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9900
[LightGBM] [Warning] min_gain_to_split is set=4.327382199822327, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.327382199822327
[LightGBM] [Warning] lambda_l2 is set=25, reg_lambda=0.0 will be ignored. Current value: lambda_l2=25
[LightGBM] [Warning] lambda_l1 is set=5, r

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:132: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:149: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)
[I 2023-04-03 14:39:37,113] Trial 98 finished with value: 27.95738964825122 and parameters: {'n_estimators': 900, 'learning_rate': 0.055594894146191016, 'num_leaves': 62, 'max_depth': 8, 'min_data_in_leaf': 9700, 'lambda_l1': 80, 'lambda_l2': 55, 'min_gain_to_split': 3.120306187193317}. Best is trial 34 with value: 27.915220176491626.
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:132: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:149: RuntimeWarning: divide by

[LightGBM] [Warning] lambda_l1 is set=80, reg_alpha=0.0 will be ignored. Current value: lambda_l1=80
[LightGBM] [Warning] min_data_in_leaf is set=9700, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9700
[LightGBM] [Warning] min_gain_to_split is set=3.120306187193317, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.120306187193317
[LightGBM] [Warning] lambda_l2 is set=55, reg_lambda=0.0 will be ignored. Current value: lambda_l2=55
[LightGBM] [Warning] lambda_l1 is set=80, reg_alpha=0.0 will be ignored. Current value: lambda_l1=80
[LightGBM] [Warning] min_data_in_leaf is set=9700, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9700
[LightGBM] [Warning] min_gain_to_split is set=3.120306187193317, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.120306187193317
[LightGBM] [Warning] lambda_l2 is set=55, reg_lambda=0.0 will be ignored. Current value: lambda_l2=55
[LightGBM] [Warning] lambda_l1 is set=

In [16]:
print('The best LightGBM paramters based on Optuna tuning are', study_lgbm.best_params)

The best LightGBM paramters based on Optuna tuning are {'n_estimators': 850, 'learning_rate': 0.0443948415663168, 'num_leaves': 55, 'max_depth': 10, 'min_data_in_leaf': 8200, 'lambda_l1': 70, 'lambda_l2': 50, 'min_gain_to_split': 3.120501829167166}


### Model 2
***
AdaBoost

In [17]:
## OPTUNA ##
def objective(trial):
    ada_params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000, step = 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3), 
    }

    model = AdaBoostRegressor(**ada_params)
    rmse_list_ada = []
    
    kf = KFold(n_splits = 10, shuffle = True)
    
    for train_ix, val_ix in kf.split(x, y):

        ## splitting the data into train and validation
        x_train_ada, x_val_ada = x.iloc[train_ix], x.iloc[val_ix]
        y_train_ada, y_val_ada = y.iloc[train_ix], y.iloc[val_ix]

        ## fitting model
        model.fit(x_train_ada, y_train_ada)
        
        ## predicting on validation
        y_pred_ada = model.predict(x_val_ada)
        
        ## appending to list
        rmse_list_ada.append(mean_squared_error(y_val_ada, y_pred_ada, squared = False))
    rmse_ada = np.mean(rmse_list_ada)
    return rmse_ada

study_ada = optuna.create_study(direction='minimize')
study_ada.optimize(objective, n_trials=100)
print('The best AdaBoost parameters based on Optuna tuning are', study_ada.best_params)

[I 2023-04-03 14:39:37,282] A new study created in memory with name: no-name-e21af334-dba7-492a-ab9b-09965c299827
[I 2023-04-03 14:39:38,272] Trial 0 finished with value: 27.99830670559544 and parameters: {'n_estimators': 750, 'learning_rate': 0.23485577614215436}. Best is trial 0 with value: 27.99830670559544.
[I 2023-04-03 14:39:39,382] Trial 1 finished with value: 27.970975940608923 and parameters: {'n_estimators': 50, 'learning_rate': 0.1020407699836876}. Best is trial 1 with value: 27.970975940608923.
[I 2023-04-03 14:39:40,387] Trial 2 finished with value: 27.94157283258395 and parameters: {'n_estimators': 650, 'learning_rate': 0.2123671399171741}. Best is trial 2 with value: 27.94157283258395.
[I 2023-04-03 14:39:41,388] Trial 3 finished with value: 27.92681536622603 and parameters: {'n_estimators': 650, 'learning_rate': 0.22086253031642353}. Best is trial 3 with value: 27.92681536622603.
[I 2023-04-03 14:39:42,172] Trial 4 finished with value: 27.950306865937126 and parameters:

The best AdaBoost parameters based on Optuna tuning are {'n_estimators': 600, 'learning_rate': 0.29871748123710284}


In [18]:
print('The best AdaBoost parameters based on Optuna tuning are', study_ada.best_params)

The best AdaBoost parameters based on Optuna tuning are {'n_estimators': 600, 'learning_rate': 0.29871748123710284}


### Model 3
***
Random Forest

In [19]:
## OPTUNA ##
def objective(trial):
    rf_params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000, step = 50),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        #"min_samples_leaf": trial.suggest_int("min_gain_to_split", 0, 1),
        #"max_leaf_nodes": trial.suggest_int("min_gain_to_split", 0, 15),
    }

    model = RandomForestRegressor(**rf_params)
    rmse_list_rf = []
    
    kf = KFold(n_splits = 10, shuffle = True)
    
    for train_ix, val_ix in kf.split(x, y):

        ## splitting the data into train and validation
        x_train_rf, x_val_rf = x.iloc[train_ix], x.iloc[val_ix]
        y_train_rf, y_val_rf = y.iloc[train_ix], y.iloc[val_ix]

        ## fitting model
        model.fit(x_train_rf, y_train_rf)
        
        ## predicting on validation
        y_pred_rf = model.predict(x_val_rf)
        
        ## appending to list
        rmse_list_rf.append(mean_squared_error(y_val_rf, y_pred_rf, squared = False))
    rmse_rf = np.mean(rmse_list_rf)
    return rmse_rf

study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective, n_trials=100)
print('The best Random Forest parameters based on Optuna tuning are', study_rf.best_params)

[I 2023-04-03 14:41:48,803] A new study created in memory with name: no-name-b5ce2dc6-ff03-414c-8a90-9cf4d3976479
[I 2023-04-03 14:42:38,357] Trial 0 finished with value: 29.380519635132003 and parameters: {'n_estimators': 900, 'max_depth': 19}. Best is trial 0 with value: 29.380519635132003.
[I 2023-04-03 14:43:00,287] Trial 1 finished with value: 28.067972485986438 and parameters: {'n_estimators': 750, 'max_depth': 6}. Best is trial 1 with value: 28.067972485986438.
[I 2023-04-03 14:43:08,691] Trial 2 finished with value: 29.470227351371413 and parameters: {'n_estimators': 150, 'max_depth': 20}. Best is trial 1 with value: 28.067972485986438.
[I 2023-04-03 14:43:15,152] Trial 3 finished with value: 28.05036679747168 and parameters: {'n_estimators': 200, 'max_depth': 7}. Best is trial 3 with value: 28.05036679747168.
[I 2023-04-03 14:43:35,446] Trial 4 finished with value: 28.72769764925247 and parameters: {'n_estimators': 450, 'max_depth': 12}. Best is trial 3 with value: 28.05036679

The best Random Forest parameters based on Optuna tuning are {'n_estimators': 550, 'max_depth': 3}


In [20]:
print('The best Random Forest parameters based on Optuna tuning are', study_rf.best_params)

The best Random Forest parameters based on Optuna tuning are {'n_estimators': 550, 'max_depth': 3}


**Because I could not get any more than two parameters to work for random forest and adaboost optuna, I have decided to do an extra model with several more parameters to tune. This may cause overfitting but I want to learn how to optuna tune a catboost model while also using random forest and adaboost frameworks**

### Model 4
***
CatBoost

In [21]:
## OPTUNA ##
def objective(trial):
    cat_params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000, step = 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3),
        "depth": trial.suggest_int("depth", 4, 10),
        "random_strength": trial.suggest_int("random_strength", 0, 10),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        
    }

    model = CatBoostRegressor(**cat_params, verbose = False)
    rmse_list_cat = []
    
    kf = KFold(n_splits = 3, shuffle = True)
    
    for train_ix, val_ix in kf.split(x, y):

        ## splitting the data into train and validation
        x_train_cat, x_val_cat = x.iloc[train_ix], x.iloc[val_ix]
        y_train_cat, y_val_cat = y.iloc[train_ix], y.iloc[val_ix]

        ## fitting model
        model.fit(x_train_cat, y_train_cat)
        
        ## predicting on validation
        y_pred_cat = model.predict(x_val_cat)
        
        ## appending to list
        rmse_list_cat.append(mean_squared_error(y_val_cat, y_pred_cat, squared = False))
    rmse_cat = np.mean(rmse_list_cat)
    return rmse_cat

study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective, n_trials=100)
print('The best CatBoost parameters based on Optuna tuning are', study_cat.best_params)

[I 2023-04-03 14:59:59,965] A new study created in memory with name: no-name-00ea1e58-93cf-4351-8967-0f62c829128d
[I 2023-04-03 15:00:01,553] Trial 0 finished with value: 28.797100941684395 and parameters: {'n_estimators': 150, 'learning_rate': 0.11155606110437429, 'depth': 8, 'random_strength': 4, 'l2_leaf_reg': 10}. Best is trial 0 with value: 28.797100941684395.
[I 2023-04-03 15:00:02,897] Trial 1 finished with value: 29.332107455979397 and parameters: {'n_estimators': 600, 'learning_rate': 0.046050934930962954, 'depth': 5, 'random_strength': 1, 'l2_leaf_reg': 3}. Best is trial 0 with value: 28.797100941684395.
[I 2023-04-03 15:00:07,136] Trial 2 finished with value: 31.12599364542176 and parameters: {'n_estimators': 250, 'learning_rate': 0.24389225380821353, 'depth': 10, 'random_strength': 10, 'l2_leaf_reg': 5}. Best is trial 0 with value: 28.797100941684395.
[I 2023-04-03 15:00:10,392] Trial 3 finished with value: 31.378447020972633 and parameters: {'n_estimators': 850, 'learning_

The best CatBoost parameters based on Optuna tuning are {'n_estimators': 50, 'learning_rate': 0.13315648268028554, 'depth': 4, 'random_strength': 7, 'l2_leaf_reg': 8}


In [22]:
print('The best CatBoost parameters based on Optuna tuning are', study_cat.best_params)

The best CatBoost parameters based on Optuna tuning are {'n_estimators': 50, 'learning_rate': 0.13315648268028554, 'depth': 4, 'random_strength': 7, 'l2_leaf_reg': 8}


### Building models with optimal hyperparameters
***

##### LGBM

In [27]:
## building lgbm model
lgbm_md = LGBMRegressor(n_estimators = 850, learning_rate = 0.0443948415663168, num_leaves = 55, max_depth = 10,
                       min_data_in_lead = 8200, lambda_l1 = 70, lambda_l2 = 50, min_gain_to_split = 3.120501829167166).fit(x, y)

## predicting on testing data
lgbm_preds = lgbm_md.predict(x_final)

[LightGBM] [Warning] Unknown parameter: min_data_in_lead
[LightGBM] [Warning] lambda_l1 is set=70, reg_alpha=0.0 will be ignored. Current value: lambda_l1=70
[LightGBM] [Warning] min_gain_to_split is set=3.120501829167166, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.120501829167166
[LightGBM] [Warning] lambda_l2 is set=50, reg_lambda=0.0 will be ignored. Current value: lambda_l2=50
914.341508179575


##### AdaBoost

In [31]:
## building adaboost model
ada_md = AdaBoostRegressor(n_estimators = 600,
                            learning_rate = 0.29871748123710284).fit(x, y)

## predicting on the testing data
ada_preds = ada_md.predict(x_final)

##### Random Forest

In [32]:
## building random forest model
rf_md = RandomForestRegressor(n_estimators = 550, max_depth = 3).fit(x, y)

## predicting on the testing data
rf_preds = rf_md.predict(x_final)

##### CatBoost

In [34]:
## buiding catboost model
cat_md = CatBoostRegressor(n_estimators = 50, learning_rate = 0.13315648268028554, depth = 4, 
                           random_strength = 7, l2_leaf_reg = 8, verbose = False).fit(x, y)

## predicting on testing data
cat_preds = cat_md.predict(x_final)

### Creating Ensemble Model
***

In [41]:
## creating dataframe to store results
ensemble_preds = pd.DataFrame()

## appending predictions to dataframe
ensemble_preds['lgbm preds'] = pd.DataFrame(lgbm_preds, columns = ['lgbm preds'])
ensemble_preds['ada preds'] = pd.DataFrame(ada_preds, columns = ['ada preds'])
ensemble_preds['rf preds'] = pd.DataFrame(rf_preds, columns = ['rf preds'])
ensemble_preds['cat preds'] = pd.DataFrame(cat_preds, columns = ['cat preds'])

## averaging the predictions
ensemble_preds['average'] = ensemble_preds.mean(axis = 1)

In [42]:
ensemble_preds.head()

,lgbm preds,ada preds,rf preds,cat preds,average
0,66.147503,53.500000,50.990152,52.810039,55.861924
1,45.917967,49.472428,49.982710,52.553042,49.481537
2,73.573751,54.233259,56.009787,54.719525,59.634080
3,43.005997,54.301887,54.751513,56.233327,52.073181
4,59.159285,49.479167,52.643024,46.964139,52.061404


In [45]:
## calculating rmse
final_rmse = mean_squared_error(y_final, ensemble_preds['average'], squared = False)

In [46]:
final_rmse

28.22449332241597

##### lgbm best rmse: **27.915220176491626**
##### adaboost best rmse: **27.88475178649582**
##### rf best rmse: **27.867010931335898**
##### catboost best rmse: **27.820507165355007**
##### ensemble model rmse: **28.22449332241597**

### Based on these results, I would utilize the CatBoost model because it had the lowest RMSE score compared to all of the other models, even the ensemble model.